# CNN Models

Created: 2019.10.21  
Notebook sequence: 9

For experimenting with keras CNN models

---

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import util
import pandas as pd
import keras
import numpy as np
from tqdm import tqdm_notebook as tqdm
from keras.layers import Dense, Conv2D, Flatten, Conv1D, Dropout
from sklearn.model_selection import train_test_split

In [3]:
data = util.load_dataset("selection_reliability_mbfc_13_w2v/sequence.pkl.pkl")
df = util.load_selection_dataset("selection_reliability_mbfc_13")

In [4]:
y = np.array(list(df.reliable))
X = np.array(data)
X_padded = keras.preprocessing.sequence.pad_sequences(X, maxlen=500, padding='post', dtype='float32')
# X = np.expand_dims(X, axis=2)
print(X_padded.shape)
print(y.shape)

(30000, 500, 300)
(30000,)


In [5]:
X_padded = np.reshape(X_padded, (X_padded.shape[0], 500*300, 1))
X_padded.shape

(30000, 150000, 1)

In [9]:
model = keras.Sequential()
#model.add(Flatten(input_shape=(500, 300, 1)))
model.add(Conv1D(16, kernel_size=300, strides=300, activation='relu', input_shape=(300*500, 1)))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(Dropout(.2))
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

adam_optimizer = keras.optimizers.Adam(learning_rate=.001, decay=1e-6)
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()
model.fit(X_padded, y, batch_size=128, validation_split=.2, epochs=100, verbose=1, use_multiprocessing=True, workers=4)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 500, 16)           4816      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 498, 16)           784       
_________________________________________________________________
dropout_1 (Dropout)          (None, 498, 16)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7968)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                255008    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 260,641
Trainable params: 260,641
Non-trainable params: 0
________________________________________________

KeyboardInterrupt: 